# Sentinel Hub Feature Info Service (FIS)

A statistical summary of satellite indices can be requested via the FIS feature. Examples of extracting time-series statistics data of Sentinel-2 and Sentinel-5P indices are provided.<br>

**Note: Cloud mask values have been included along with the obtained statistics values of Sentinel-2 indices in the following examples.**

In [35]:
# import libraries
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from shapely.geometry import Polygon
import geopandas as gpd

from sentinelhub import FisRequest, BBox, Geometry, CRS, WcsRequest, CustomUrlParam, \
    DataCollection, HistogramType, bbox_to_dimensions
from sentinelhub.time_utils import iso_to_datetime

In [36]:
# sentinel hub configurations
from sentinelhub import SHConfig


INSTANCE_ID = ''  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

## Parameters of FIS request


* `layer` -   name of the layer defined in Sentinel Hub Configurator.
* `geometry_list` - list of geometry objects (BBox or Geometry), statistics will be calculated for each of them separately
* `time` - statistics will be calculated for each acquisition in the give time interval separately 
* `resolution` - spatial resolution on which to calculate statistics 
* `data_folder` - optional parameter for specifying location where the data should be saved locally 
* `bins` - The number of bins (a positive integer) in the histogram. When this parameter is absent, no histogram is computed.
* `histogram_type` - ways of dividing values into bins currently supported: EQUIDISTANT, EQUALFREQUENCY, or STREAMING

In [37]:
def fis_data_to_dataframe(fis_data):
    """ Creates a DataFrame from list of FIS responses
    """
    COLUMNS = ['channel', 'date', 'min', 'max', 'mean', 'stDev']
    data = []

    for fis_response in fis_data:
        for channel, channel_stats in fis_response.items():
            for stat in channel_stats:
                row = [int(channel[1:]), iso_to_datetime(stat['date'])]

                for column in COLUMNS[2:]:
                    row.append(stat['basicStats'][column])

                data.append(row)

    return pd.DataFrame(data, columns=COLUMNS).sort_values(['channel', 'date'])

In [47]:
# NDVI request
def ndvi_request(city_shp, start_date, end_date, city_name):
    geometry = Geometry(city_shp.geometry.values[0], crs=CRS.WGS84)
    time_interval = (start_date, end_date)
    ndvi_stats_fis = FisRequest(
        data_collection=DataCollection.SENTINEL2_L2A,
        layer='NDVI',
        geometry_list=[geometry],
        time=time_interval,
        resolution='100m',
        bins=20, # remove this if distribution values not needed
        histogram_type=HistogramType.EQUIDISTANT, # remove this if distribution values not needed
        config=config
    )
    ndvi_stats = ndvi_stats_fis.get_data()  
    ndvi_stats_df = clean_df(ndvi_stats)
    ndvi_stats_df.to_csv(f'ndvi-{city_name}.csv')
    return ndvi_stats_df

# NDBI request
def ndbi_request(city_shp, start_date, end_date, city_name):
    geometry = Geometry(city_shp.geometry.values[0], crs=CRS.WGS84)
    time_interval = (start_date, end_date)
    ndbi_stats_fis = FisRequest(
        data_collection=DataCollection.SENTINEL2_L2A,
        layer='NDBI',
        geometry_list=[geometry],
        time=time_interval,
        resolution='100m',
        bins=20, # remove this if distribution values not needed
        histogram_type=HistogramType.EQUIDISTANT, # remove this if distribution values not needed
        config=config
    )
    ndbi_stats = ndbi_stats_fis.get_data()  
    ndbi_stats_df = clean_df(ndbi_stats)
    ndbi_stats_df.to_csv(f'ndbi-{city_name}.csv')
    return ndbi_stats_df

# NO2 request
def no_request(city_shp, start_date, end_date, city_name):
    geometry = Geometry(city_shp.geometry.values[0], crs=CRS.WGS84)
    time_interval = (start_date, end_date)
    no_stats_fis = FisRequest(
        data_collection=DataCollection.SENTINEL5P,
        layer='NO2',
        geometry_list=[geometry],
        time=time_interval,
        resolution='100m',
        config=config
    )
    no_stats = no_stats_fis.get_data()  
    no_stats_df = clean_df(no_stats)
    no_stats_df.to_csv(f'no2-{city_name}.csv')
    return no_stats_df

In [48]:
# convert to clean df ready for analysis
def clean_df (json_stats):
    raw_df_stats = fis_data_to_dataframe(json_stats).reset_index(drop=True)
    index_df = raw_df_stats[raw_df_stats.channel ==0]
    cloud_df = raw_df_stats[raw_df_stats.channel ==1]
    cloud_df = cloud_df.drop(['channel'], axis=1).add_suffix('_clm')
    merged_df = pd.merge(index_df, cloud_df, left_on ='date', right_on = 'date_clm', how='outer')
    clean_merged_df = merged_df.drop(['channel', 'date_clm'], axis=1)
    return clean_merged_df

In [49]:
# load shapefiles of COE cities

# manila city
manila = gpd.read_file('coe-cities-shp/manila.shp')

# angeles city
angeles = gpd.read_file('coe-cities-shp/Angeles.shp')

# bacolod city
bacolod = gpd.read_file('coe-cities-shp/Bacolod.shp')

# davao
davao = gpd.read_file('coe-cities-shp/Davao.shp')

# cebu
cebu = gpd.read_file('coe-cities-shp/Cebu.shp')

# iloilo city
iloilo = gpd.read_file('coe-cities-shp/Iloilo.shp')

## Manila City

In [50]:
# ndbi
ndbi_request(manila,'2021-8-01', '2021-9-30','manila')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-08-05,-0.891345,-0.124911,-0.478799,0.138289,1.0,1.0,1.000000,0.000000
1,2021-08-10,-0.553011,-0.032954,-0.374115,0.052897,1.0,1.0,1.000000,0.000000
2,2021-08-15,-0.643073,-0.347226,-0.511861,0.058119,1.0,1.0,1.000000,0.000000
3,2021-08-20,-0.647975,0.396337,-0.037584,0.174471,0.0,1.0,0.538009,0.498553
4,2021-08-25,-1.000000,1.000000,-0.140904,0.090328,0.0,1.0,0.989109,0.103790
5,2021-08-30,-0.294238,0.220428,-0.016633,0.074853,1.0,1.0,1.000000,0.000000
6,2021-09-04,-1.000000,0.465967,0.043789,0.119823,0.0,1.0,0.719887,0.449054
7,2021-09-09,-1.000000,1.000000,0.063890,0.128195,0.0,1.0,0.644958,0.478526
8,2021-09-14,-1.000000,1.000000,0.074155,0.177837,0.0,1.0,0.493792,0.499961
9,2021-09-19,-0.687299,-0.641544,-0.667657,0.006843,1.0,1.0,1.000000,0.000000


In [18]:
# ndvi
ndvi_request(manila,'2021-9-01', '2021-9-30','manila')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-04,-0.867965,0.895870,0.056187,0.139108,0.0,1.0,0.719887,0.449054
1,2021-09-09,-1.000000,0.951219,0.088570,0.171981,0.0,1.0,0.644958,0.478526
2,2021-09-14,-1.000000,1.000000,0.078586,0.201346,0.0,1.0,0.493792,0.499961
3,2021-09-19,0.005269,0.017466,0.011025,0.001579,1.0,1.0,1.000000,0.000000
4,2021-09-24,-1.000000,1.000000,0.064444,0.201715,0.0,1.0,0.624047,0.484368
5,2021-09-29,-1.000000,1.000000,0.090397,0.189781,0.0,1.0,0.322588,0.467466


In [20]:
# no2
no_request(manila,'2021-9-01', '2021-9-30','manila')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-03,0.000061,0.000091,0.000071,0.00001,NaN,NaN,NaN,NaN
3,2021-09-04,0.000022,0.000113,0.000086,0.000014,NaN,NaN,NaN,NaN
4,2021-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-09-06,0.000092,0.000154,0.000109,0.000028,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-09,0.000035,0.000116,0.000057,0.000025,NaN,NaN,NaN,NaN
9,2021-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Angeles City 

In [51]:
# ndbi
ndbi_request(angeles,'2021-8-01', '2021-9-30','angeles')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-08-05,-0.879838,-0.381187,-0.566224,0.072882,1.0,1.0,1.000000,0.000000
1,2021-08-10,-0.373430,0.309008,-0.116900,0.072558,1.0,1.0,1.000000,0.000000
2,2021-08-15,-0.808348,-0.559567,-0.713109,0.041759,1.0,1.0,1.000000,0.000000
3,2021-08-20,-0.454494,0.167552,-0.279520,0.076190,1.0,1.0,1.000000,0.000000
4,2021-08-25,-0.630367,1.000000,-0.125516,0.141358,0.0,1.0,0.758202,0.428172
5,2021-08-30,-0.450753,0.446263,-0.148551,0.126617,0.0,1.0,0.976850,0.150380
6,2021-09-04,-0.534650,0.124814,-0.240541,0.067104,1.0,1.0,1.000000,0.000000
7,2021-09-09,-0.790262,1.000000,-0.126611,0.159829,0.0,1.0,0.706075,0.455558
8,2021-09-14,-1.000000,1.000000,-0.133229,0.156758,0.0,1.0,0.858785,0.348243
9,2021-09-19,-0.593930,-0.404216,-0.511678,0.049336,1.0,1.0,1.000000,0.000000


In [21]:
ndvi_request(angeles,'2021-9-01', '2021-9-30','angeles')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-04,-0.069606,0.709073,0.048487,0.102065,1.0,1.0,1.000000,0.000000
1,2021-09-09,-1.000000,1.000000,0.447985,0.301096,0.0,1.0,0.706075,0.455558
2,2021-09-14,-1.000000,1.000000,0.321208,0.322330,0.0,1.0,0.858785,0.348243
3,2021-09-19,0.028836,0.094625,0.050240,0.008088,1.0,1.0,1.000000,0.000000
4,2021-09-24,-1.000000,1.000000,0.493889,0.339191,0.0,1.0,0.356590,0.478992
5,2021-09-29,-1.000000,1.000000,0.277212,0.263154,1.0,1.0,1.000000,0.000000


In [22]:
no_request(angeles,'2021-9-01', '2021-9-30','angeles')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,0.000014,0.000022,0.000019,0.000004,NaN,NaN,NaN,NaN
1,2021-09-02,0.000025,0.000039,0.000034,0.000005,NaN,NaN,NaN,NaN
2,2021-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-09-06,0.000014,0.000026,0.000019,0.000004,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-09,0.000026,0.00003,0.000029,0.000002,NaN,NaN,NaN,NaN
9,2021-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Bacolod City 

In [52]:
# ndbi
ndbi_request(bacolod,'2021-8-01', '2021-9-30','bacolod')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-08-02,-0.989943,1.000000,-0.174450,0.142878,0.0,1.0,0.811313,0.391260
1,2021-08-07,-1.000000,1.000000,-0.145395,0.176593,0.0,1.0,0.361358,0.480394
2,2021-08-12,-0.952466,1.000000,-0.155018,0.144444,0.0,1.0,0.659958,0.473723
3,2021-08-17,-0.644420,1.000000,-0.163437,0.118459,0.0,1.0,0.750402,0.432780
4,2021-08-22,-0.592304,0.371573,-0.237154,0.077168,0.0,1.0,0.984936,0.121808
5,2021-08-27,-1.000000,1.000000,-0.147398,0.134130,0.0,1.0,0.700625,0.457984
6,2021-09-01,-0.661726,-0.472185,-0.593372,0.030894,1.0,1.0,1.000000,0.000000
7,2021-09-06,-0.548337,0.217404,-0.168287,0.077253,1.0,1.0,1.000000,0.000000
8,2021-09-11,-0.492485,0.002760,-0.246517,0.058209,1.0,1.0,1.000000,0.000000
9,2021-09-16,-0.817782,-0.578674,-0.719827,0.034369,1.0,1.0,1.000000,0.000000


In [23]:
ndvi_request(bacolod,'2021-9-01', '2021-9-30','bacolod')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,-0.019377,0.158739,0.042353,0.025252,1.0,1.0,1.000000,0.000000
1,2021-09-06,-0.099526,0.612202,0.163654,0.121225,1.0,1.0,1.000000,0.000000
2,2021-09-11,-0.063448,0.596961,0.258928,0.124082,1.0,1.0,1.000000,0.000000
3,2021-09-16,-0.038218,0.044600,-0.003801,0.005972,1.0,1.0,1.000000,0.000000
4,2021-09-21,-1.000000,1.000000,0.386964,0.285163,0.0,1.0,0.592974,0.491280
5,2021-09-26,-0.352851,1.000000,0.307264,0.245830,0.0,1.0,0.868770,0.337651


In [24]:
no_request(bacolod,'2021-9-01', '2021-9-30','bacolod')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-03,-0.00002,0.000027,0.000004,0.000012,NaN,NaN,NaN,NaN
3,2021-09-04,0.000006,0.000014,0.000007,0.000002,NaN,NaN,NaN,NaN
4,2021-09-05,-0.0,0.000016,0.000008,0.000005,NaN,NaN,NaN,NaN
5,2021-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Davao City 

In [53]:
# ndbi
ndbi_request(davao,'2021-8-01', '2021-9-30','davao')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-08-04,-1.000000,1.000000,-0.263927,0.114212,0.0,1.0,0.542983,0.498149
1,2021-08-09,-0.670514,-0.495334,-0.592053,0.027310,1.0,1.0,1.000000,0.000000
2,2021-08-14,-0.645805,1.000000,-0.237425,0.104767,0.0,1.0,0.699607,0.458429
3,2021-08-19,-1.000000,1.000000,-0.223180,0.153978,0.0,1.0,0.512882,0.499834
4,2021-08-24,-0.728696,0.218157,-0.314778,0.066547,0.0,1.0,0.994560,0.073555
5,2021-08-29,-1.000000,1.000000,-0.218754,0.145540,0.0,1.0,0.577497,0.493958
6,2021-09-03,-1.000000,1.000000,-0.234607,0.150256,0.0,1.0,0.256451,0.436674
7,2021-09-08,-0.795250,1.000000,-0.212679,0.117207,0.0,1.0,0.633383,0.481881
8,2021-09-13,-1.000000,1.000000,-0.220894,0.135150,0.0,1.0,0.412552,0.492294
9,2021-09-18,-0.810018,1.000000,-0.214397,0.076814,0.0,1.0,0.868193,0.338281


In [25]:
ndvi_request(davao,'2021-9-01', '2021-9-30','davao')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-03,-1.00000,1.000000,0.641166,0.307220,0.0,1.0,0.256451,0.436674
1,2021-09-08,-1.00000,1.000000,0.406729,0.315084,0.0,1.0,0.633383,0.481881
2,2021-09-13,-1.00000,1.000000,0.577012,0.319720,0.0,1.0,0.412552,0.492294
3,2021-09-18,-1.00000,1.000000,0.326152,0.223645,0.0,1.0,0.868193,0.338281
4,2021-09-23,-1.00000,1.000000,0.672307,0.268197,0.0,1.0,0.162187,0.368622
5,2021-09-28,-0.10119,0.498342,0.193001,0.076511,1.0,1.0,1.000000,0.000000


In [27]:
no_request(davao,'2021-9-01', '2021-9-30','davao')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,-0.000018,0.000071,0.000013,0.000013,NaN,NaN,NaN,NaN
1,2021-09-02,-0.000013,0.000021,0.000004,0.000008,NaN,NaN,NaN,NaN
2,2021-09-03,-0.000015,0.000032,0.000006,0.00001,NaN,NaN,NaN,NaN
3,2021-09-04,-0.000001,0.000029,0.000011,0.000008,NaN,NaN,NaN,NaN
4,2021-09-05,-0.000003,0.000024,0.000013,0.000008,NaN,NaN,NaN,NaN
5,2021-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-09-07,0.000012,0.000012,0.000012,0.0,NaN,NaN,NaN,NaN
7,2021-09-08,0.000008,0.00001,0.000008,0.0,NaN,NaN,NaN,NaN
8,2021-09-09,0.000009,0.000009,0.000009,0.0,NaN,NaN,NaN,NaN
9,2021-09-10,0.000017,0.000022,0.000021,0.000002,NaN,NaN,NaN,NaN


## Cebu City 

In [54]:
# ndbi
ndbi_request(cebu,'2021-8-01', '2021-9-30','cebu')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-08-02,-0.880597,0.698312,-0.154114,0.130716,0.0,1.0,0.878078,0.327195
1,2021-08-07,-0.828010,1.000000,-0.216683,0.168429,0.0,1.0,0.033361,0.179578
2,2021-08-12,-1.000000,1.000000,-0.182807,0.175836,0.0,1.0,0.546850,0.497800
3,2021-08-17,-0.809881,-0.204301,-0.465821,0.083838,1.0,1.0,1.000000,0.000000
4,2021-08-22,-0.528160,1.000000,-0.333534,0.159589,0.0,1.0,0.965933,0.181401
5,2021-08-27,-0.587331,0.145444,-0.214576,0.084897,1.0,1.0,1.000000,0.000000
6,2021-09-01,-0.858505,-0.095537,-0.442674,0.132605,1.0,1.0,1.000000,0.000000
7,2021-09-06,-0.591018,-0.323587,-0.482836,0.054317,1.0,1.0,1.000000,0.000000
8,2021-09-11,-0.399603,0.059084,-0.265646,0.063337,1.0,1.0,1.000000,0.000000
9,2021-09-16,-0.866598,0.759003,-0.335075,0.133820,0.0,1.0,0.999261,0.027177


In [28]:
ndvi_request(cebu,'2021-9-01', '2021-9-30','cebu')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,-0.297740,0.205635,0.016945,0.028910,1.0,1.0,1.000000,0.000000
1,2021-09-06,-0.031729,0.039608,-0.000643,0.010155,1.0,1.0,1.000000,0.000000
2,2021-09-11,-0.109656,0.406885,0.164722,0.084196,1.0,1.0,1.000000,0.000000
3,2021-09-16,-0.740299,1.000000,0.018586,0.105855,0.0,1.0,0.999261,0.027177
4,2021-09-21,-1.000000,1.000000,0.421859,0.330314,0.0,1.0,0.630405,0.482695
5,2021-09-26,-1.000000,1.000000,0.297564,0.241832,0.0,1.0,0.940803,0.235993


In [29]:
no_request(cebu,'2021-9-01', '2021-9-30','cebu')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-02,0.000005,0.000023,0.000016,0.000005,NaN,NaN,NaN,NaN
2,2021-09-03,0.000005,0.000006,0.000005,0.0,NaN,NaN,NaN,NaN
3,2021-09-04,0.000002,0.000013,0.000013,0.000002,NaN,NaN,NaN,NaN
4,2021-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,0.000001,0.00003,0.000017,0.000008,NaN,NaN,NaN,NaN
8,2021-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-09-10,0.000009,0.000009,0.000009,0.0,NaN,NaN,NaN,NaN


## Iloilo City 

In [51]:
# ndbi
ndbi_request(iloilo,'2021-8-01', '2021-9-30','angeles')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-08-05,-0.879838,-0.381187,-0.566224,0.072882,1.0,1.0,1.000000,0.000000
1,2021-08-10,-0.373430,0.309008,-0.116900,0.072558,1.0,1.0,1.000000,0.000000
2,2021-08-15,-0.808348,-0.559567,-0.713109,0.041759,1.0,1.0,1.000000,0.000000
3,2021-08-20,-0.454494,0.167552,-0.279520,0.076190,1.0,1.0,1.000000,0.000000
4,2021-08-25,-0.630367,1.000000,-0.125516,0.141358,0.0,1.0,0.758202,0.428172
5,2021-08-30,-0.450753,0.446263,-0.148551,0.126617,0.0,1.0,0.976850,0.150380
6,2021-09-04,-0.534650,0.124814,-0.240541,0.067104,1.0,1.0,1.000000,0.000000
7,2021-09-09,-0.790262,1.000000,-0.126611,0.159829,0.0,1.0,0.706075,0.455558
8,2021-09-14,-1.000000,1.000000,-0.133229,0.156758,0.0,1.0,0.858785,0.348243
9,2021-09-19,-0.593930,-0.404216,-0.511678,0.049336,1.0,1.0,1.000000,0.000000


In [30]:
ndvi_request(iloilo,'2021-9-01', '2021-9-30','iloilo')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,0.004310,0.237923,0.070716,0.044908,1.0,1.0,1.000000,0.000000
1,2021-09-06,-0.009813,0.490488,0.212438,0.081883,1.0,1.0,1.000000,0.000000
2,2021-09-11,-0.033231,0.173814,0.018666,0.035883,1.0,1.0,1.000000,0.000000
3,2021-09-16,-0.031662,0.033374,0.002751,0.004615,1.0,1.0,1.000000,0.000000
4,2021-09-21,-1.000000,1.000000,0.227327,0.292402,0.0,1.0,0.748073,0.434120
5,2021-09-26,-1.000000,1.000000,0.321156,0.328449,0.0,1.0,0.623546,0.484496


In [31]:
no_request(iloilo,'2021-9-01', '2021-9-30','iloilo')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,0.000007,0.000037,0.000021,0.000008,NaN,NaN,NaN,NaN
1,2021-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-03,0.000004,0.000004,0.000004,0.0,NaN,NaN,NaN,NaN
3,2021-09-04,-0.000004,0.000016,0.000001,0.000008,NaN,NaN,NaN,NaN
4,2021-09-05,0.000012,0.000025,0.000019,0.000006,NaN,NaN,NaN,NaN
5,2021-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
